### Prompt:
You are an analyst for a fastfood restaurant. The company has tasked you to analyze their past transactions. Each row represents a single transaction/order from one of their ten (10) branches. The order is recorded as a __`string`__ which is formatted as: 

"`[quantity1][item_code1], [quantity2][item_code2], [quantity3][item_code3]`"

However, because of some technical error, the __number of combo meals purchased__ as well as the __total order value__ was lost, so we'll need to restore that.

### You are tasked to come up with the following:
1. Given the order column, count how many of each product was purchased
2. Given each product count, get the number of supposed combo meals purchased. <br><i>(You may update the product count when the combo meals are counted)</i>.
3. Calculate the total order value.
4. Determine which branch made the most revenue, and how much?

### The combo meals are detailed below:
1. Family combo: 4 burgers, 4 fries, 4 drinks
2. Big combo: 1 burgers, 1 fries, 1 drinks
3. Snack combo: 1 fries, 1 drinks

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("food_transactions.csv")
df2 = pd.read_json("price_list.json")

df

,Unnamed: 0,date,branch,order
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink"
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink"
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink"
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink"
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink"
...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink"
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries"
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries"
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink"


In [4]:
# 1. Given the order column, count how many of each product was purchased

df['order_list'] = df['order'].apply(lambda x: str(x).split(", "))
df['burgers'] = df['order_list'].apply(lambda x: int(x[0][0]) if "S_Burger" in x[0] else 0)

df['order_noburgers'] = df['order_list'].apply(lambda x: x[1:] if "S_Burger" in x[0] else x)
df['fries'] = df['order_noburgers'].apply(lambda x: 0 if len(x) == 0 else int(x[0][0]) if "S_Fries" in x[0] else 0)

df['order_nofries'] = df['order_noburgers'].apply(lambda x: [] if len(x) == 0 else x[1:] if "S_Fries" in x[0] else x)
df['drinks'] = df['order_nofries'].apply(lambda x: 0 if len(x) == 0 else int(x[0][0]) if x[0][0] != 'n' else 0)

df

,Unnamed: 0,date,branch,order,order_list,burgers,order_noburgers,fries,order_nofries,drinks
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink","[1S_Burger, 4S_Fries, 5S_Drink]",1,"[4S_Fries, 5S_Drink]",4,[5S_Drink],5
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink","[2S_Burger, 4S_Fries, 3S_Drink]",2,"[4S_Fries, 3S_Drink]",4,[3S_Drink],3
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink","[2S_Burger, 4S_Fries, 2S_Drink]",2,"[4S_Fries, 2S_Drink]",4,[2S_Drink],2
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink","[6S_Fries, 1S_Drink]",0,"[6S_Fries, 1S_Drink]",6,[1S_Drink],1
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink","[5S_Burger, 6S_Drink]",5,[6S_Drink],0,[6S_Drink],6
...,...,...,...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink","[3S_Fries, 6S_Drink]",0,"[3S_Fries, 6S_Drink]",3,[6S_Drink],6
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries","[7S_Burger, 6S_Fries]",7,[6S_Fries],6,[],0
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries","[4S_Burger, 6S_Fries]",4,[6S_Fries],6,[],0
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink","[7S_Burger, 6S_Fries, 5S_Drink]",7,"[6S_Fries, 5S_Drink]",6,[5S_Drink],5


In [12]:
# 2. Given each product count, get the number of supposed combo meals purchased.
# (You may update the product count when the combo meals are counted).
# The combo meals are detailed below:
# Family combo: 4 burgers, 4 fries, 4 drinks
# Big combo: 1 burgers, 1 fries, 1 drinks
# Snack combo: 1 fries, 1 drinks


family_combo = df[(df['burgers'] % 4 == 0) & (df['fries'] % 4 == 0) & (df['drinks'] % 4 == 0)].count()['order']
family_combo

big_combo = df[(df['burgers'] % 4 != 0) & (df['fries'] % 4 != 0) & (df['drinks'] % 4 != 0) & (df['burgers'] == df['fries']) & (df['fries'] == df['drinks'])].count()['order']

big_combo_df = df[(df['burgers'] % 4 != 0) & (df['fries'] % 4 != 0) & (df['drinks'] % 4 != 0) & (df['burgers'] == df['fries']) & (df['fries'] == df['drinks'])]

df['big combo count'] = df.apply(lambda x: x['burgers'] % 4 if x['burgers'] == x['fries'] and x['burgers'] == x['drinks'] else 0, axis=1)
df['family combo count'] = df.apply(lambda x: x['burgers'] // 4 if x['burgers'] == x['fries'] and x['burgers'] == x['drinks'] else 0, axis=1)
df['snack combo'] = df.apply(lambda x: x['fries'] if x['burgers'] == 0 and x['drinks'] == x['fries'] and x['big combo count'] == 0 else 0, axis=1)

df['snack combo'].sum() + df['big combo count'].sum() + df['family combo count'].sum()

df


,Unnamed: 0,date,branch,order,order_list,burgers,order_noburgers,fries,order_nofries,drinks,big combo count,family combo count,snack combo,big combo price,family combo price,price_nocombos,snack combo price,total revenue
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink","[1S_Burger, 4S_Fries, 5S_Drink]",1,"[4S_Fries, 5S_Drink]",4,[5S_Drink],5,0,0,0,0,0,610,0,610
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink","[2S_Burger, 4S_Fries, 3S_Drink]",2,"[4S_Fries, 3S_Drink]",4,[3S_Drink],3,0,0,0,0,0,601,0,601
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink","[2S_Burger, 4S_Fries, 2S_Drink]",2,"[4S_Fries, 2S_Drink]",4,[2S_Drink],2,0,0,0,0,0,552,0,552
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink","[6S_Fries, 1S_Drink]",0,"[6S_Fries, 1S_Drink]",6,[1S_Drink],1,0,0,0,0,0,463,0,463
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink","[5S_Burger, 6S_Drink]",5,[6S_Drink],0,[6S_Drink],6,0,0,0,0,0,739,0,739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink","[3S_Fries, 6S_Drink]",0,"[3S_Fries, 6S_Drink]",3,[6S_Drink],6,0,0,0,0,0,501,0,501
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries","[7S_Burger, 6S_Fries]",7,[6S_Fries],6,[],0,0,0,0,0,0,1037,0,1037
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries","[4S_Burger, 6S_Fries]",4,[6S_Fries],6,[],0,0,0,0,0,0,770,0,770
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink","[7S_Burger, 6S_Fries, 5S_Drink]",7,"[6S_Fries, 5S_Drink]",6,[5S_Drink],5,0,0,0,0,0,1282,0,1282


In [10]:
# 3. Calculate the total order value.

dict = df2.to_dict()

df['big combo price'] = df['big combo count']*int(dict['price']['Big_combo'])

df['family combo price'] = df['family combo count']*int(dict['price']['Fam_combo'])

df['snack combo price'] = df['snack combo']*int(dict['price']['Snack_combo'])

df['price_nocombos'] =  df['burgers']*int(dict['price']['S_Burger']) + df['fries']*int(dict['price']['S_Fries']) + df['drinks']*int(dict['price']['S_Drink'])

df['total revenue'] = df['big combo price'] + df['family combo price'] + df['snack combo price'] + df['price_nocombos'] 

df

,Unnamed: 0,date,branch,order,order_list,burgers,order_noburgers,fries,order_nofries,drinks,big combo count,family combo count,snack combo,big combo price,family combo price,price_nocombos,snack combo price,total revenue
0,0,2023-06-26 21:36:39,Branch 3,"1S_Burger, 4S_Fries, 5S_Drink","[1S_Burger, 4S_Fries, 5S_Drink]",1,"[4S_Fries, 5S_Drink]",4,[5S_Drink],5,0,0,0,0,0,610,0,610
1,1,2023-12-22 10:57:56,Branch 9,"2S_Burger, 4S_Fries, 3S_Drink","[2S_Burger, 4S_Fries, 3S_Drink]",2,"[4S_Fries, 3S_Drink]",4,[3S_Drink],3,0,0,0,0,0,601,0,601
2,2,2023-12-14 14:07:51,Branch 8,"2S_Burger, 4S_Fries, 2S_Drink","[2S_Burger, 4S_Fries, 2S_Drink]",2,"[4S_Fries, 2S_Drink]",4,[2S_Drink],2,0,0,0,0,0,552,0,552
3,3,2023-11-01 02:10:22,Branch 4,"6S_Fries, 1S_Drink","[6S_Fries, 1S_Drink]",0,"[6S_Fries, 1S_Drink]",6,[1S_Drink],1,0,0,0,0,0,463,0,463
4,4,2023-09-30 07:42:12,Branch 3,"5S_Burger, 6S_Drink","[5S_Burger, 6S_Drink]",5,[6S_Drink],0,[6S_Drink],6,0,0,0,0,0,739,0,739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
134770,134770,2023-12-11 04:21:04,Branch 8,"3S_Fries, 6S_Drink","[3S_Fries, 6S_Drink]",0,"[3S_Fries, 6S_Drink]",3,[6S_Drink],6,0,0,0,0,0,501,0,501
134771,134771,2023-08-25 18:43:05,Branch 8,"7S_Burger, 6S_Fries","[7S_Burger, 6S_Fries]",7,[6S_Fries],6,[],0,0,0,0,0,0,1037,0,1037
134772,134772,2023-09-24 14:15:58,Branch 10,"4S_Burger, 6S_Fries","[4S_Burger, 6S_Fries]",4,[6S_Fries],6,[],0,0,0,0,0,0,770,0,770
134773,134773,2023-07-08 12:43:10,Branch 4,"7S_Burger, 6S_Fries, 5S_Drink","[7S_Burger, 6S_Fries, 5S_Drink]",7,"[6S_Fries, 5S_Drink]",6,[5S_Drink],5,0,0,0,0,0,1282,0,1282


In [13]:
# 4. Determine which branch made the most revenue, and how much?

filter = df.groupby('branch').sum()['total revenue']

new_table = pd.DataFrame(filter)

print(new_table)

#See non-existent table haha

           total revenue
branch                  
Branch 1       799706987
Branch 10      774473500
Branch 2       715206085
Branch 3       725972286
Branch 4       727032490
Branch 5       724847768
Branch 6       730808664
Branch 7       788023176
Branch 8       712288635
Branch 9       692048629
